In [1]:
import os
import numpy as np
import random
from scipy import ndimage
from skimage import transform
import keras
from keras.layers import Dense, Lambda
from keras.applications import VGG16
from keras.models import Model
from keras.optimizers import SGD
from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping
import skimage.io as io

# MEAN = np.array([123.0, 117.0, 104.0])
MEAN = np.array([0, 0, 0])
def getModel( output_dim ):
    ''' 
        * output_dim: the number of classes (int)
        
        * return: compiled model (keras.engine.training.Model)
    '''
    vgg_model = VGG16( weights='imagenet', include_top=True )
#     vgg_out = vgg_model.layers[-2].output #Last FC layer's output 
    layer_name = 'fc2'
    x = vgg_model.get_layer(layer_name).output
    temper = 2.0;
    x = Lambda(lambda x: x/temper)(x)
    
    layer_name = 'predictions'
    pred_layer = vgg_model.get_layer(layer_name) 
    weight = pred_layer.get_weights()
    x = Dense(1000, activation='softmax', name='softmax1', weights=weight)(x) 
    
    softmax_layer = Dense(256, activation='softmax')(x) #Create softmax layer taking input as vgg_ou
    #Create new transfer learning model
    tl_model = Model( input=vgg_model.input, output=softmax_layer )

    #Freeze all layers of VGG16 and Compile the model
    #Confirm the model is appropriate
    for l in vgg_model.layers:
        l.trainable = False
        
    layer_name = 'softmax1'
    tl_model.get_layer(layer_name).trainable = False
#     tl_model.get_layer(layer_name).set_weights(weights)
    
    tl_model.summary()

    return tl_model
def convert_image(collect):
#     sz = np.shape(collect)[0]
    sz = len(collect)
    images = np.zeros((sz, 224, 224, 3))
    for i in range(sz):
        img = collect[i]
        img = transform.resize(img, (224, 224))*255
        if (len(img.shape) != 3):
           img = img[:,:,np.newaxis]
        images[i] = img
    return images

sizeOfEachClass = 16
sizeOfTestEachClass = 16
path='/home/yiluo/Homework/253/3/256_ObjectCategories/'
files = os.listdir(path)
X_train, X_val, X_test = [], [], []
Y_train, Y_val, Y_test = [], [], []
label = 0
for category in files:
    sub_path = path + category
    sub_files = os.listdir(sub_path)
    cnt = np.shape(sub_files)[0]
    num = 1
    for img_nm in sub_files:
        if img_nm[-4:] == '.jpg':
            img_path = sub_path + '/' + img_nm
            #img = ndimage.imread(sub_path + '/' + img_nm)
            #img = transform.resize(img, (224, 224))*255
            if num <= np.floor(0.8*cnt):
                if num <= sizeOfEachClass:
                    X_train.append(img_path)
                    Y_train.append(label)
            elif num <= np.floor(0.9*cnt):
                if num <= np.floor(0.8*cnt) + sizeOfTestEachClass:
                    X_val.append(img_path)
                    Y_val.append(label)
            else:
                X_test.append(img_path)
                Y_test.append(label)
            num += 1
    label += 1

Using Theano backend.
Using gpu device 0: GeForce GTX 1080 (CNMeM is disabled, cuDNN not available)


In [2]:
label

256

In [3]:
#shuffle
ind=range(int(len(X_train)))
random.shuffle(ind)
X_train = [X_train[i] for i in ind]
Y_train = [Y_train[i] for i in ind]
Y_train_label = np.zeros((len(Y_train), 256))
for i in range(len(Y_train)):
    Y_train_label[i, Y_train[i]] = 1
    
#validation  
Y_val_label = np.zeros((len(Y_val), 256))
for i in range(len(Y_val)):
    Y_val_label[i, Y_val[i]] = 1
    
#test
Y_test_label = np.zeros((len(Y_test), 256))
for i in range(len(Y_test)):
    Y_test_label[i, Y_test[i]] = 1

In [4]:
np.shape(Y_train_label)

(4096, 256)

In [5]:
# #Output dim for your dataset
# output_dim = 256 #For Caltech256

# tl_model = getModel( output_dim )
# tl_model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])
# for epoch in range(10):
#     start = 0
#     for batch in range(len(X_train)/64):
#         coll=io.ImageCollection(X_train[start:start+64])
#         images = convert_image(coll)
#         tl_model.train_on_batch(images, Y_train_label[start:start+64, :])
# #         tl_model.summary()
#         start += 64
# print 'stop here'

In [6]:
#Output dim for your dataset
output_dim = 256 #For Caltech256

#Data
coll = io.ImageCollection(X_train)
X_train_data = convert_image(coll)
# coll_val = io.ImageCollection(X_val)
# X_val_data = convert_image(coll_val)

In [7]:
#Model
tl_model = getModel( output_dim )
# rmsprop = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0) # error

# tl_model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

/usr/local/lib/python2.7/dist-packages/Keras-1.2.2-py2.7.egg/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_1 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 4096)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
block1_conv1 (Convolution2D)     (None, 224, 224, 64)  1792        input_1[0][0]                    
____________________________________________________________________________________________________
block1_conv2 (Convolution2D)     (None, 224, 224, 64)  36928       block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_pool (MaxPooling2D)       (None, 112, 112, 64)  0           block1_conv2[0][0]               
___________________________________________________________________________________________

In [8]:
# sgd = keras.optimizers.SGD(lr=5, momentum=0.9, decay=0.0, nesterov=True)
# tl_model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

adagrad = keras.optimizers.Adagrad(lr=1, epsilon=1e-08, decay=0.0)
tl_model.compile(loss='categorical_crossentropy', optimizer=adagrad, metrics=['accuracy'])

In [9]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    samplewise_center=True)

# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
datagen.fit(X_train_data)

In [10]:
hist = tl_model.fit(X_train_data, Y_train_label, 
             batch_size=32, nb_epoch=20, 
             verbose=1, validation_split=0.5,
             shuffle=None, class_weight=None, sample_weight=None)

Train on 2048 samples, validate on 2048 samples
Epoch 1/20
2048/2048 [==============================] - 85s - loss: 4.5117 - acc: 0.1768 - val_loss: 3.5688 - val_acc: 0.2954
Epoch 2/20
2048/2048 [==============================] - 82s - loss: 2.6515 - acc: 0.5405 - val_loss: 3.0968 - val_acc: 0.3906
Epoch 3/20
2048/2048 [==============================] - 82s - loss: 2.0846 - acc: 0.6914 - val_loss: 2.8808 - val_acc: 0.4297
Epoch 4/20
2048/2048 [==============================] - 82s - loss: 1.7498 - acc: 0.7764 - val_loss: 2.7572 - val_acc: 0.4541
Epoch 5/20
2048/2048 [==============================] - 82s - loss: 1.5187 - acc: 0.8247 - val_loss: 2.6783 - val_acc: 0.4624
Epoch 6/20
2048/2048 [==============================] - 84s - loss: 1.3462 - acc: 0.8628 - val_loss: 2.6245 - val_acc: 0.4697
Epoch 7/20
2048/2048 [==============================] - 84s - loss: 1.2111 - acc: 0.8936 - val_loss: 2.5863 - val_acc: 0.4800
Epoch 8/20
2048/2048 [==============================] - 47s - loss: 1.

In [11]:
print(hist.history)

{'acc': [0.1767578125, 0.54052734375, 0.69140625, 0.7763671875, 0.82470703125, 0.86279296875, 0.8935546875, 0.9130859375, 0.92529296875, 0.93896484375, 0.9501953125, 0.955078125, 0.962890625, 0.96728515625, 0.97314453125, 0.9755859375, 0.978515625, 0.98046875, 0.98388671875, 0.986328125], 'loss': [4.5116996094584465, 2.6514799743890762, 2.0846369434148073, 1.7498182319104671, 1.518721804022789, 1.3462304715067148, 1.2111167497932911, 1.1017184155061841, 1.0109664145857096, 0.93426722194999456, 0.86847683414816856, 0.81135548744350672, 0.76125565450638533, 0.71693347301334143, 0.67742934171110392, 0.64198922971263528, 0.61001156084239483, 0.58100964175537229, 0.55458520818501711, 0.53040876891463995], 'val_acc': [0.29541015625, 0.390625, 0.4296875, 0.4541015625, 0.46240234375, 0.4697265625, 0.47998046875, 0.4833984375, 0.48681640625, 0.48681640625, 0.48876953125, 0.49169921875, 0.4931640625, 0.49365234375, 0.49365234375, 0.49609375, 0.49609375, 0.49658203125, 0.49755859375, 0.4990234375

In [9]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=2, verbose=0, mode='auto'),
    ModelCheckpoint(filepath="./weights.hdf5", verbose=1, save_best_only=True)
]

In [31]:
# tl_model.fit(X_train_data, Y_train_label,
#           nb_epoch=20,
#           batch_size=16)

tl_model.fit(X_train_data, Y_train_label, 
             batch_size=32, nb_epoch=1, 
             verbose=1,  
             shuffle=None, class_weight=None, sample_weight=None)

Epoch 1/1
4096/4096 [==============================] - 40s - loss: 0.2756 - acc: 0.9353    


In [33]:
score = tl_model.evaluate(X_val_data, Y_val_label, batch_size=32, verbose=1, sample_weight=None)
print score

2695/2695 [==============================] - 26s    
[1.7824576142546418, 0.59962894253031884]


In [ ]:
import sys

# manual batch
nb_epoch = 20
nm_per_batch = 32
for e in range(nb_epoch):
    print 'Epoch', e
    batches = 0
    for X_batch, Y_batch in datagen.flow(X_train_data, Y_train_label, batch_size=nm_per_batch, shuffle=True):
        loss = tl_model.train_on_batch(X_batch, Y_batch)
#         print 'batches:', batches, ', loss:' , loss[0] , ', acc train:' , loss[1]
#         print 'batches:', batches, 'loss:' , loss[0],
        print 'batches:', batches,
        
        batches += 1
        if batches >= len(X_train) / nm_per_batch:
            # we need to break the loop by hand because
            # the generator loops indefinitely
            break
    
    #evaluate on validation
    score = tl_model.evaluate(X_val_data, Y_val_label, batch_size=nm_per_batch, verbose=1, sample_weight=None)
    print 'loss:' , loss[0] , ', acc train:' , loss[1], 'loss train: ', score